In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime ,time

In [3]:
# rumor reply tweets
reply = pd.read_csv(r"D:\論文\PHEME9\Code\sydneysiege\18 targetanalysis\targetdata\sydneysiege_target_reply_entire.csv", encoding = 'utf-8', header = 0,index_col=0)
reply.head(1)

,src_tweet_id,reply_tweet_id,reply_user_id,reply_tweet,reply_date,rumorlabel,cleaned_reply_tw,reply_sentiment,targetlabel,src_user_id,src_tweet,src_date
0,544267656597995521,544269152198721536,443573208,@KristyMayr7 that is not an IS flag specifically.,Sun Dec 14 23:14:12 +0000 2014,1,not flag specifically,Neutral,0,258627226,BREAKING: Hostages are being forced to hold an...,Sun Dec 14 23:08:15 +0000 2014


In [4]:
print(reply.shape)
print(reply.isnull().sum())

(7371, 12)
src_tweet_id        0
reply_tweet_id      0
reply_user_id       0
reply_tweet         0
reply_date          0
rumorlabel          0
cleaned_reply_tw    0
reply_sentiment     0
targetlabel         0
src_user_id         0
src_tweet           0
src_date            0
dtype: int64


In [5]:
reply.targetlabel.value_counts()

0    5428
1    1943
Name: targetlabel, dtype: int64

In [6]:
reply.reply_date = pd.to_datetime(reply.reply_date)

In [7]:
reply.sort_values(by='reply_date',inplace=True)
reply.reset_index(inplace=True,drop=True)
reply

,src_tweet_id,reply_tweet_id,reply_user_id,reply_tweet,reply_date,rumorlabel,cleaned_reply_tw,reply_sentiment,targetlabel,src_user_id,src_tweet,src_date
0,544268637134393344,544269047076896769,2313204931,“@9NewsSyd: UPDATE: An #ISIS flag is being dis...,2014-12-14 23:13:47+00:00,1,jesus christ,Neutral,0,171802941,UPDATE: An ISIS flag is being displayed in the...,Sun Dec 14 23:12:09 +0000 2014
1,544267656597995521,544269152198721536,443573208,@KristyMayr7 that is not an IS flag specifically.,2014-12-14 23:14:12+00:00,1,not flag specifically,Neutral,0,258627226,BREAKING: Hostages are being forced to hold an...,Sun Dec 14 23:08:15 +0000 2014
2,544268637134393344,544269246201475072,25172734,@9NewsSyd holy lord! Prayers to all. This is t...,2014-12-14 23:14:34+00:00,1,holy lord prayers much,Neutral,0,171802941,UPDATE: An ISIS flag is being displayed in the...,Sun Dec 14 23:12:09 +0000 2014
3,544268637134393344,544269317718544384,17984709,"@9NewsSyd So Islamists no threat to Australia,...",2014-12-14 23:14:52+00:00,1,islamists no threat australia right,Positive,1,171802941,UPDATE: An ISIS flag is being displayed in the...,Sun Dec 14 23:12:09 +0000 2014
4,544268637134393344,544269468482801665,1687941666,"@LillSaleh Yup, every news source I have is ru...",2014-12-14 23:15:27+00:00,1,yup every news source run timeline break tv soon,Neutral,0,171802941,UPDATE: An ISIS flag is being displayed in the...,Sun Dec 14 23:12:09 +0000 2014
...,...,...,...,...,...,...,...,...,...,...,...,...
7366,544305913260433408,545329080858460161,553010243,@NumbatMusings @PzFeed @CDoranHarader You didn...,2014-12-17 21:25:59+00:00,1,didnt like us citizens self protection guy see...,Negative,0,292777349,BREAKING UPDATE: US Consulate in Sydney has be...,Mon Dec 15 01:40:17 +0000 2014
7367,544305913260433408,545329446019141632,553010243,"@NumbatMusings @PzFeed @CDoranHarader Also, Ho...",2014-12-17 21:27:26+00:00,1,also attacker legal gun not allow plus lot con...,Negative,1,292777349,BREAKING UPDATE: US Consulate in Sydney has be...,Mon Dec 15 01:40:17 +0000 2014
7368,544305913260433408,545329782767251456,553010243,@NumbatMusings @PzFeed @CDoranHarader At Least...,2014-12-17 21:28:46+00:00,1,least america lawful self defense gun dont cri...,Positive,1,292777349,BREAKING UPDATE: US Consulate in Sydney has be...,Mon Dec 15 01:40:17 +0000 2014
7369,544409517224898560,545373022958260224,36738874,@ErwinPoverty @FoxNews I'm sure there were adu...,2014-12-18 00:20:35+00:00,1,im sure adults smart ass,Positive,0,1367531,#Sydneysiege latest: Gunman reportedly demands...,Mon Dec 15 08:31:58 +0000 2014


In [8]:
target_index = reply.loc[reply.targetlabel ==1].index
print("number of targeted source tweets: ",len(target_index))
# target_index

number of targeted source tweets:  1943


# Calculate sentiment changes before target

In [9]:
# create "before" dictionary to store sentiments of non-targeting replys,
# where has three values: ratios of three sentiments/ negative to positive ratio / 
# and indices of those non-targeting replys
j = 0
before = {"sentiment": [],"np_ratio" :[],"index":[]}
target = list()
for i in target_index:
  if j == i: #the next source tweet also had target label == 1 
    before['sentiment'].append(np.nan)
    before['np_ratio'].append(np.nan)
    before['index'].append(np.nan)
    b = reply.loc[i,'reply_sentiment']
    target.append(b)
    
  else:     
    count = reply.loc[j:i-1,'reply_sentiment'].value_counts(normalize=True,sort=False)
    # ratio of three sentiments
    sen = count.round(2).to_dict()
    before['sentiment'].append(sen)
    #ratio of Negative to Positive
    try:
      neg = count['Negative']
      pos = count['Positive']
    except:
      if 'Positive' in count.index:
        # some positive tws but no negative ones
        n_p = 0
      elif "Negative" in count.index:
        # some negative tws but no positive ones
        n_p = np.inf
      else:
        # only neutral tws
        n_p = 0 
    else:
      n_p = round(neg/pos,2)
    finally:
      before['np_ratio'].append(n_p)
    before['index'].append([*range(j,i)])
    # target's sentiment
    b = reply.loc[i,'reply_sentiment']
    target.append(b)

  j =i+1

In [10]:
pd.set_option('display.max_colwidth', None)
sentiment_change = pd.DataFrame({"sentiment_beforetarget" : before['sentiment'],
                                 "neg_to_pos_beforetarget" : before['np_ratio'],
                                 "sentiment_target" : target,
                                 "target_index": target_index})


# Calculate sentiment changes after target arrived until the next one came.

In [11]:
# calculte the sentiment changes after the "last" targeing reply arrive.
# save it into "last_value" variable.

sentiment_aftertarget = sentiment_change['sentiment_beforetarget']
last_value = reply.loc[target_index[-1]+1: ,'reply_sentiment'].value_counts(normalize=True,sort=False).round(2).to_dict()
sentiment_aftertarget = pd.concat([sentiment_aftertarget[1:] ,pd.Series([last_value])], ignore_index=True )
sentiment_aftertarget

0                         {'Neutral': 0.67, 'Negative': 0.33}
1                                                         NaN
2                                           {'Negative': 1.0}
3                                            {'Neutral': 1.0}
4       {'Positive': 0.33, 'Negative': 0.33, 'Neutral': 0.33}
                                ...                          
1938                                                      NaN
1939                                                      NaN
1940                     {'Positive': 0.25, 'Negative': 0.75}
1941                                                      NaN
1942                        {'Positive': 0.5, 'Neutral': 0.5}
Length: 1943, dtype: object

In [12]:
sentiment_change['sentiment_aftertarget'] = sentiment_aftertarget
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,sentiment_target,target_index,sentiment_aftertarget
0,{'Neutral': 1.0},0.0,Positive,3,"{'Neutral': 0.67, 'Negative': 0.33}"
1,"{'Neutral': 0.67, 'Negative': 0.33}",inf,Neutral,7,NaN
2,NaN,NaN,Neutral,8,{'Negative': 1.0}
3,{'Negative': 1.0},inf,Positive,10,{'Neutral': 1.0}
4,{'Neutral': 1.0},0.0,Negative,12,"{'Positive': 0.33, 'Negative': 0.33, 'Neutral': 0.33}"
...,...,...,...,...,...
1938,"{'Neutral': 0.5, 'Negative': 0.5}",inf,Neutral,7360,NaN
1939,NaN,NaN,Negative,7361,NaN
1940,NaN,NaN,Negative,7362,"{'Positive': 0.25, 'Negative': 0.75}"
1941,"{'Positive': 0.25, 'Negative': 0.75}",3.0,Negative,7367,NaN


In [13]:
try:
    neg = last_value['Negative']
    pos = last_value['Positive']
except:
  if 'Positive' in last_value.keys():
    n_p = 0
  elif "Negative" in last_value.keys():
    n_p = np.inf
  else:
    n_p = 0
else:
  n_p = round(neg/pos,2)



# neg = last_value['Negative']
# pos = last_value['Positive'] 
# n_p = round(neg/pos,2)

neg_to_pos_aftertarget = sentiment_change['neg_to_pos_beforetarget'][1:]
neg_to_pos_aftertarget = pd.concat([neg_to_pos_aftertarget, pd.Series([n_p])], ignore_index=True )

sentiment_change['neg_to_pos_aftertarget'] = neg_to_pos_aftertarget
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,sentiment_target,target_index,sentiment_aftertarget,neg_to_pos_aftertarget
0,{'Neutral': 1.0},0.0,Positive,3,"{'Neutral': 0.67, 'Negative': 0.33}",inf
1,"{'Neutral': 0.67, 'Negative': 0.33}",inf,Neutral,7,NaN,NaN
2,NaN,NaN,Neutral,8,{'Negative': 1.0},inf
3,{'Negative': 1.0},inf,Positive,10,{'Neutral': 1.0},0.0
4,{'Neutral': 1.0},0.0,Negative,12,"{'Positive': 0.33, 'Negative': 0.33, 'Neutral': 0.33}",1.0
...,...,...,...,...,...,...
1938,"{'Neutral': 0.5, 'Negative': 0.5}",inf,Neutral,7360,NaN,NaN
1939,NaN,NaN,Negative,7361,NaN,NaN
1940,NaN,NaN,Negative,7362,"{'Positive': 0.25, 'Negative': 0.75}",3.0
1941,"{'Positive': 0.25, 'Negative': 0.75}",3.0,Negative,7367,NaN,NaN


## calculate number of rows

In [14]:
# calculate number of rows before the target sentiment arrived.
row_before = []
for i in range(len(target_index)):
  if i >0 :
    row_before.append(target_index[i] - target_index[i-1]-1)
  else:
    row_before.append(target_index[i])
print(row_before)


[3, 3, 0, 1, 1, 9, 5, 0, 1, 0, 7, 3, 8, 4, 1, 0, 2, 0, 0, 3, 5, 4, 2, 9, 5, 2, 3, 1, 1, 0, 9, 2, 0, 6, 0, 0, 6, 10, 0, 3, 0, 0, 3, 1, 1, 1, 1, 2, 0, 0, 2, 2, 5, 0, 1, 8, 4, 1, 0, 0, 5, 1, 2, 11, 2, 0, 3, 6, 6, 4, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1, 0, 3, 1, 0, 2, 0, 0, 4, 3, 1, 0, 4, 1, 3, 3, 0, 0, 0, 2, 1, 0, 1, 0, 1, 1, 3, 15, 0, 0, 0, 2, 10, 1, 6, 8, 3, 12, 4, 0, 3, 0, 6, 14, 0, 4, 2, 0, 1, 0, 2, 4, 0, 0, 0, 1, 2, 1, 3, 1, 1, 0, 0, 5, 0, 1, 1, 3, 0, 0, 1, 9, 3, 3, 2, 5, 6, 1, 4, 1, 0, 1, 0, 1, 6, 1, 3, 0, 9, 0, 0, 2, 0, 2, 1, 0, 4, 2, 4, 0, 1, 0, 2, 6, 1, 6, 0, 0, 5, 2, 0, 3, 0, 2, 0, 3, 1, 0, 1, 2, 0, 6, 6, 0, 2, 1, 2, 1, 3, 3, 3, 2, 4, 1, 7, 2, 4, 2, 3, 4, 3, 3, 1, 0, 1, 2, 12, 0, 0, 0, 4, 2, 3, 4, 8, 9, 2, 4, 1, 3, 2, 4, 3, 1, 1, 0, 7, 4, 0, 1, 1, 16, 0, 2, 0, 3, 1, 1, 1, 0, 3, 0, 0, 2, 0, 8, 2, 3, 0, 3, 2, 1, 3, 0, 1, 3, 9, 1, 9, 2, 1, 0, 0, 3, 5, 5, 5, 3, 7, 11, 4, 2, 0, 3, 1, 0, 1, 8, 5, 2, 5, 4, 1, 0, 4, 0, 1, 0, 1, 0, 5, 5, 0, 4, 0, 5, 2, 5, 6, 1, 3, 19, 16, 1, 15, 1, 0, 

In [15]:
# calculate number of rows after the target sentiment arrived.
row_after = []
for i in range(len(target_index)):
  if i+1 < len(target_index):
    row_after.append(target_index[i+1] - target_index[i]-1)
  else: #last one
    row_after.append(reply.index[-1]-target_index[i])
print(row_after)


[3, 0, 1, 1, 9, 5, 0, 1, 0, 7, 3, 8, 4, 1, 0, 2, 0, 0, 3, 5, 4, 2, 9, 5, 2, 3, 1, 1, 0, 9, 2, 0, 6, 0, 0, 6, 10, 0, 3, 0, 0, 3, 1, 1, 1, 1, 2, 0, 0, 2, 2, 5, 0, 1, 8, 4, 1, 0, 0, 5, 1, 2, 11, 2, 0, 3, 6, 6, 4, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1, 0, 3, 1, 0, 2, 0, 0, 4, 3, 1, 0, 4, 1, 3, 3, 0, 0, 0, 2, 1, 0, 1, 0, 1, 1, 3, 15, 0, 0, 0, 2, 10, 1, 6, 8, 3, 12, 4, 0, 3, 0, 6, 14, 0, 4, 2, 0, 1, 0, 2, 4, 0, 0, 0, 1, 2, 1, 3, 1, 1, 0, 0, 5, 0, 1, 1, 3, 0, 0, 1, 9, 3, 3, 2, 5, 6, 1, 4, 1, 0, 1, 0, 1, 6, 1, 3, 0, 9, 0, 0, 2, 0, 2, 1, 0, 4, 2, 4, 0, 1, 0, 2, 6, 1, 6, 0, 0, 5, 2, 0, 3, 0, 2, 0, 3, 1, 0, 1, 2, 0, 6, 6, 0, 2, 1, 2, 1, 3, 3, 3, 2, 4, 1, 7, 2, 4, 2, 3, 4, 3, 3, 1, 0, 1, 2, 12, 0, 0, 0, 4, 2, 3, 4, 8, 9, 2, 4, 1, 3, 2, 4, 3, 1, 1, 0, 7, 4, 0, 1, 1, 16, 0, 2, 0, 3, 1, 1, 1, 0, 3, 0, 0, 2, 0, 8, 2, 3, 0, 3, 2, 1, 3, 0, 1, 3, 9, 1, 9, 2, 1, 0, 0, 3, 5, 5, 5, 3, 7, 11, 4, 2, 0, 3, 1, 0, 1, 8, 5, 2, 5, 4, 1, 0, 4, 0, 1, 0, 1, 0, 5, 5, 0, 4, 0, 5, 2, 5, 6, 1, 3, 19, 16, 1, 15, 1, 0, 0, 

In [16]:
sentiment_change['count_of_rows_bef'] = pd.Series(row_before)
sentiment_change['count_of_rows_after'] = pd.Series(row_after)

In [17]:
sentiment_change.columns

Index(['sentiment_beforetarget', 'neg_to_pos_beforetarget', 'sentiment_target',
       'target_index', 'sentiment_aftertarget', 'neg_to_pos_aftertarget',
       'count_of_rows_bef', 'count_of_rows_after'],
      dtype='object')

In [18]:
sentiment_change = sentiment_change[['sentiment_beforetarget', 'neg_to_pos_beforetarget','count_of_rows_bef',  
                                    'sentiment_target',
                                    'sentiment_aftertarget', 'neg_to_pos_aftertarget','count_of_rows_after']]
sentiment_change

,sentiment_beforetarget,neg_to_pos_beforetarget,count_of_rows_bef,sentiment_target,sentiment_aftertarget,neg_to_pos_aftertarget,count_of_rows_after
0,{'Neutral': 1.0},0.0,3,Positive,"{'Neutral': 0.67, 'Negative': 0.33}",inf,3
1,"{'Neutral': 0.67, 'Negative': 0.33}",inf,3,Neutral,NaN,NaN,0
2,NaN,NaN,0,Neutral,{'Negative': 1.0},inf,1
3,{'Negative': 1.0},inf,1,Positive,{'Neutral': 1.0},0.0,1
4,{'Neutral': 1.0},0.0,1,Negative,"{'Positive': 0.33, 'Negative': 0.33, 'Neutral': 0.33}",1.0,9
...,...,...,...,...,...,...,...
1938,"{'Neutral': 0.5, 'Negative': 0.5}",inf,2,Neutral,NaN,NaN,0
1939,NaN,NaN,0,Negative,NaN,NaN,0
1940,NaN,NaN,0,Negative,"{'Positive': 0.25, 'Negative': 0.75}",3.0,4
1941,"{'Positive': 0.25, 'Negative': 0.75}",3.0,4,Negative,NaN,NaN,0


# save file

In [21]:
savepath = r'sydneysiege\18 targetanalysis\twanalysis\reply_target_sentiment.csv'

with open(savepath, 'w', encoding = 'utf-8') as f:
  sentiment_change.to_csv(f)